# Benchmark: Koalas (PySpark) and Dask - Data Preparation
The benchmark was performed against the 2009 - 2013 Yellow Taxi Trip Records (157 GB) from NYC Taxi and Limousine Commission (TLC) Trip Record Data.

The CSV files were downloaded into Databricks File System (DBFS), and then were converted into Parquet files via Koalas for better efficiency.

Download url: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page.

Data dictionary: https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf.

The scenario used in this benchmark was inspired by https://github.com/xdssio/big_data_benchmarks.

# Download CSV files to DBFS

In [1]:
%%sh
sudo mkdir -p ../../datasets/taxi_csv/

In [2]:
%%sh
ls ../../datasets/taxi_csv

yellow_taxi_tripdata_2011-01.parquet
yellow_taxi_tripdata_2011-02.parquet
yellow_taxi_tripdata_2011-03.parquet
yellow_taxi_tripdata_2011-04.parquet
yellow_taxi_tripdata_2011-05.parquet
yellow_taxi_tripdata_2011-06.parquet
yellow_taxi_tripdata_2011-07.parquet
yellow_taxi_tripdata_2011-08.parquet
yellow_taxi_tripdata_2011-09.parquet
yellow_taxi_tripdata_2011-10.parquet
yellow_taxi_tripdata_2011-11.parquet
yellow_taxi_tripdata_2011-12.parquet
yellow_taxi_tripdata_2012-01.parquet
yellow_taxi_tripdata_2012-02.parquet
yellow_taxi_tripdata_2012-03.parquet
yellow_taxi_tripdata_2012-04.parquet
yellow_taxi_tripdata_2012-05.parquet
yellow_taxi_tripdata_2012-06.parquet
yellow_taxi_tripdata_2012-07.parquet
yellow_taxi_tripdata_2012-08.parquet
yellow_taxi_tripdata_2012-09.parquet
yellow_taxi_tripdata_2012-10.parquet
yellow_taxi_tripdata_2012-11.parquet
yellow_taxi_tripdata_2012-12.parquet
yellow_taxi_tripdata_2013-01.parquet
yellow_taxi_tripdata_2013-02.parquet
yellow_taxi_tripdata_2013-03.parquet
y

The old download version didn't work, changed it to a new one using "download_taxi_datasets.py"

In [ ]:
# url_loc = {} # Map download url to the file location in DBFS

# for year in range(2009, 2014):
#   for m in range(1, 13):
#     month = "{:02d}".format(m)
#     fname = 'yellow_tripdata_%s-%s.csv' % (year, month)
#     url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/%s' % fname
#     loc = f'{DATASETS_DIR}/taxi_csv/%s' % fname
#     url_loc[url] = loc

In [ ]:
# import urllib.request

# for url, loc in url_loc.items():
#   urllib.request.urlretrieve(url, loc)

Using our version to download data

<span style="font-size:20pt"><b>NOTE:</b> </span> the dataset changed so we had to change the years of this analysis... Initialy this was between 2009 and 2014, now it's from 2011 until 2015... (The columns changed from 2011 forward)

In [3]:
import datetime as dt
import os

# Add "code" folder to path so we can import the "download_taxi_files" script
import sys
sys.path.insert(1, os.path.abspath('..'))
from download_taxi_datasets import download_taxi_files    

DATASETS_DIR = '../../datasets'
years = range(2011, 2013)
date_range = [dt.datetime(year, month, 1) for year in years for month in range(1, 13)]
files = download_taxi_files(date_range, f'{DATASETS_DIR}/taxi_csv')

100%|██████████| 60/60 [00:00<00:00, 1973.56it/s]


In [4]:
%%sh
ls ../../datasets/taxi_csv

yellow_taxi_tripdata_2011-01.parquet
yellow_taxi_tripdata_2011-02.parquet
yellow_taxi_tripdata_2011-03.parquet
yellow_taxi_tripdata_2011-04.parquet
yellow_taxi_tripdata_2011-05.parquet
yellow_taxi_tripdata_2011-06.parquet
yellow_taxi_tripdata_2011-07.parquet
yellow_taxi_tripdata_2011-08.parquet
yellow_taxi_tripdata_2011-09.parquet
yellow_taxi_tripdata_2011-10.parquet
yellow_taxi_tripdata_2011-11.parquet
yellow_taxi_tripdata_2011-12.parquet
yellow_taxi_tripdata_2012-01.parquet
yellow_taxi_tripdata_2012-02.parquet
yellow_taxi_tripdata_2012-03.parquet
yellow_taxi_tripdata_2012-04.parquet
yellow_taxi_tripdata_2012-05.parquet
yellow_taxi_tripdata_2012-06.parquet
yellow_taxi_tripdata_2012-07.parquet
yellow_taxi_tripdata_2012-08.parquet
yellow_taxi_tripdata_2012-09.parquet
yellow_taxi_tripdata_2012-10.parquet
yellow_taxi_tripdata_2012-11.parquet
yellow_taxi_tripdata_2012-12.parquet
yellow_taxi_tripdata_2013-01.parquet
yellow_taxi_tripdata_2013-02.parquet
yellow_taxi_tripdata_2013-03.parquet
y

In [5]:
# Changed bellow section to work in a normal Linux machine
# total_bytes = 0
# for fileInfo in dbutils.fs.ls(f'{OUTPUT_FOLDER}/taxi_csv'):
#   total_bytes += fileInfo.size
import os
total_bytes = 0
for filename in os.listdir(f'{DATASETS_DIR}/taxi_csv'):
    file_path = os.path.join(f'{DATASETS_DIR}/taxi_csv', filename)
    if os.path.isfile(file_path): total_bytes += os.path.getsize(file_path)
print('%s GBs data in total' % (total_bytes * 1e-9))

10.711220603000001 GBs data in total


# Convert to Parquet files
Convert downloaded CSV files into Parquet files via Koalas for better efficiency.

In [6]:
# import databricks.koalas as ks
 
# ks.set_option('compute.default_index_type', 'distributed-sequence')

# Instead of using koalas to rename the columns, we're going to use dask
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=4, threads_per_worker=3, memory_limit='30GiB')
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 120.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45833,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 120.00 GiB
Comm: tcp://127.0.0.1:34549,Total threads: 3
Dashboard: http://127.0.0.1:38675/status,Memory: 30.00 GiB
Nanny: tcp://127.0.0.1:41857,


## Changing columns names

In order for the datasets to match the format of the ones in the article, we had to change some of the column names

In [7]:
renaming_cols = {
  'passenger_count': 'Passenger_Count',
  'fare_amount': 'Fare_Amt',
  'tip_amount': 'Tip_Amt',
  # The 2 renamings bellow, don't show the exact LAT/LON, 
  # but we only want to benchmark, so they're not entirely relevant...
  # We'll later create the respective Lat coordinates by copying the values of this lon ones
  'PULocationID': 'Start_Lon',
  'DOLocationID': 'End_Lon',

  'tolls_amount': 'Tolls_Amt',
  'total_amount': 'Total_Amt',
}

In [8]:
dtype_dict = {
  'Passenger_Count': 'int64', 
  'Start_Lon': 'float64', 
  'Start_Lat': 'float64',
  'End_Lon': 'float64', 
  'End_Lat': 'float64', 
  'Fare_Amt': 'float64', 
  'Tip_Amt': 'float64', 
  'Tolls_Amt': 'float64',
  'Total_Amt': 'float64'
}
# Convert Article dtype_dict to new one
dtype_dict = {col: dtype_dict[renaming_cols[col]] for col in renaming_cols.keys()}
ks_df = dd.read_parquet(f'{DATASETS_DIR}/taxi_csv', dtype=dtype_dict)

# Drop all other columns not relevant (to us)
ks_df = ks_df.drop(columns=ks_df.columns.difference(dtype_dict.keys()))

In [9]:
# Renaming columns to match old format
ks_df = ks_df.rename(columns=renaming_cols)

# Create the Lat/Lon columns
ks_df['Start_Lat'] = ks_df['Start_Lon'].copy()
ks_df['End_Lat'] = ks_df['End_Lon'].copy()

ks_df.columns = ks_df.columns.str.lower()

In [10]:
%%sh
rm -fr ../../datasets/ks_taxi_parquet

In [11]:
ks_df.to_parquet(f'{DATASETS_DIR}/ks_taxi_parquet')

In [12]:
# total_bytes = 0
# for file_info in dbutils.fs.ls('FileStore/ks_taxi_parquet'):
#   total_bytes += file_info.size
# print('%s GBs data in total' % (total_bytes * 1e-9))

total_bytes = 0
for filename in os.listdir(f'{DATASETS_DIR}/ks_taxi_parquet'):
    file_path = os.path.join(f'{DATASETS_DIR}/ks_taxi_parquet', filename)
    if os.path.isfile(file_path): total_bytes += os.path.getsize(file_path)
print('%s GBs data in total' % (total_bytes * 1e-9))

10.723366313000001 GBs data in total


# Note: Filtering Size
(Size of filtered data / Size of total data) in the benchmark

In [13]:
import databricks.koalas as ks
koalas_data = ks.read_parquet(f'{DATASETS_DIR}/ks_taxi_parquet')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/18 08:36:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
expr_filter = (koalas_data.tip_amt >= 1) & (koalas_data.tip_amt <= 5)
 
print(f'In the benchmark, filtered data is {len(koalas_data[expr_filter]) / len(koalas_data) * 100}% of total data')

In the benchmark, filtered data is 45.102692369637836% of total data
